# Imports

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Data

$ 
y0 = 3.4  x_{0} - 1.2 x_{2} \\
y1 = 2.7 x_{1} + 4.5
$

In [3]:
n_data = 20
n_f = 3
n_o = 2

x = np.random.uniform(-10, 10, (n_data, n_f))
w0 = np.array([[3.4, 0, -1.2], [0,2.7,0]])
b0 = np.array([0, 4.5])
t = np.dot(x, w0.T) + b0

x = torch.tensor(x)
t = torch.tensor(t)

# Creating a linear regression from scratch

In [4]:
class Linear_Regression:
    def __init__(self, n_features, n_outputs) -> None:
        self.n_f = n_features
        self.n_o = n_outputs
        self.w = torch.rand((self.n_o, self.n_f), dtype=torch.float64, requires_grad=True)
        self.b = torch.rand(self.n_o, dtype=torch.float64, requires_grad=True)

    def predict(self, x):
        return torch.matmul(x, self.w.T) + self.b

    def MSE(self, t, y):
        diff = t - y
        return torch.sum(diff * diff) / diff.numel()

    def train(self, x, t, n_epochs, lr):
        for e in range(n_epochs):
            y = self.predict(x)
            loss = self.MSE(t, y)
            loss.backward()
            with torch.no_grad():
                self.w -= self.w.grad * lr
                self.b -= self.b.grad * lr
                self.w.grad.zero_()
                self.b.grad.zero_()

    def show_weights(self):
        print("equation")

        for i in range(self.n_o):
            equation = "    y[{}] = ".format(i)
            for j in range(self.n_f):
                op = "+"
                if self.w[i][j] < 0:
                    op = "-"

                equation += " {}{:.4f} * x[{}]".format(op, torch.abs(self.w[i][j]), j)

            op = "+"
            if self.b[i] < 0:
                op = "-"
                
            
            equation += " {}{:.4f}".format(op, torch.abs(self.b[i]))
        
            print(equation)


In [5]:
LR = Linear_Regression(3, 2)
LR.train(x,t, 1000, 0.001)
LR.show_weights()

equation
    y[0] =  +3.4224 * x[0] -0.0110 * x[1] -1.1896 * x[2] +0.2702
    y[1] =  -0.1691 * x[0] +2.7828 * x[1] -0.0782 * x[2] +2.4618


# Linear Regression by PyTorch

In [6]:
train_ds = TensorDataset(x, t)
train_dl = DataLoader(train_ds, 5, shuffle=True)

In [7]:
class Linear_Regression_PyTorch:
    def __init__(self, n_features, n_outputs) -> None:
        self.n_f = n_features
        self.n_o = n_outputs
        self.model = nn.Linear(self.n_f, self.n_o, dtype=torch.float64)


    def predict(self, x):
        return self.model(x)


    def train(self, x, t, n_epochs, batch_size, lr):
        train_ds = TensorDataset(x, t)
        train_dl = DataLoader(train_ds, batch_size, shuffle=True)
        opt = torch.optim.SGD(self.model.parameters(), lr=lr)

        for e in range(n_epochs):
            for xs, ts in train_dl:
                ys = self.model(xs)
                loss = torch.nn.functional.mse_loss(ys, ts)
                loss.backward()
                opt.step()
                opt.zero_grad()

            if (e + 1) % (n_epochs/10) == 0:
                print('Epoch [{}/{}], Loss: {:.4f}'.format(e+1, n_epochs, loss.item()))


    def show_weights(self):
        print("equation")
        w = self.model.weight
        b = self.model.bias

        for i in range(self.n_o):
            equation = "    y[{}] =".format(i)
            for j in range(self.n_f):
                op = "+"
                if w[i][j] < 0:
                    op = "-"

                equation += " {}{:.4f} * x[{}]".format(op, torch.abs(w[i][j]), j)

            op = "+"
            if b[i] < 0:
                op = "-"
                
            
            equation += " {}{:.4f}".format(op, torch.abs(b[i]))
        
            print(equation)

In [11]:
LRP = Linear_Regression_PyTorch(3,2)
LRP.train(x,t,10000,20,0.001)
LRP.show_weights()

Epoch [1000/10000], Loss: 1.2569
Epoch [2000/10000], Loss: 0.3063
Epoch [3000/10000], Loss: 0.0747
Epoch [4000/10000], Loss: 0.0182
Epoch [5000/10000], Loss: 0.0044
Epoch [6000/10000], Loss: 0.0011
Epoch [7000/10000], Loss: 0.0003
Epoch [8000/10000], Loss: 0.0001
Epoch [9000/10000], Loss: 0.0000
Epoch [10000/10000], Loss: 0.0000
equation
    y[0] = +3.4000 * x[0] -0.0000 * x[1] -1.2000 * x[2] +0.0003
    y[1] = -0.0003 * x[0] +2.7001 * x[1] -0.0001 * x[2] +4.4967
